# Hands-on: Ziffernklassifizierung (MNIST)

In diesem Hands-On werden wir ein neuronale Netz trainieren, das handschriftliche Ziffern erkennt. Basis ist die MNIST Database (Modified National Institute of Standards and Technology database). MNIST wurde im Jahr 1998 veröffentlich und ist seitdem ein Klassiker des Maschinellen Lernens und wird in vielen Kursen zum Einstieg in die Verarbeitung und Klassifikation von Bildern verwendet. Auch in wissenschaftlichen Veröffentlichungen ist es immer noch ein Standard, an dem viele Verfahren (insbesondere aus dem Bereich der Bildverarbeitung) beweisen müssen.

In diesem Notebook werden wir ein einfaches neuronales Netzwerk bauen, das eine gute Leistung auf MNIST erzielt. Dies zeigt zugleich die Fortschritte und das Potenzial des Deep Learnings. Die hier mit wenigen Zeilen Code erreichten Ergebnisse sind viel besser, als sie selbst die ausgefeiltesten traditionallen Verfahren noch vor wenigen Jahren erreichten.

## Allgemeine Einstellungen

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings("ignore")

import tensorflow
tensorflow.compat.v1.logging.set_verbosity(tensorflow.compat.v1.logging.ERROR)

## Daten besorgen

Entsprechend seinem Referenzstatus wird MNIST direkt durch die Keras-Bibliothek zur Verfügung gestellt und kann mit einer Codezeile eingelesen werden

In [ ]:
import keras.datasets.mnist

(raw_train_images, raw_train_labels), (raw_test_images, raw_test_labels) = keras.datasets.mnist.load_data()

In [ ]:
raw_train_labels

In [ ]:
print("Struktur der Labels:", raw_train_labels.shape)
print("Struktur der Bilddaten:", raw_train_images.shape)

-> Die Trainingsdaten umfassen 60.000 Ziffern, die jeweils aus 28x28 Grauwerten bestehen

Wir schauen uns nun eines der Bilder an. Schaut euch auch ein paar andere Beispiele an, indem ihr den Index (Variable `example_nr`) anpasst. Was fällt euch z.B. beim Beispiel mit dem Index 42 auf? Als welche Ziffer hättet ihr das Bild klassifiziert?

In [ ]:
import matplotlib.pyplot
%matplotlib inline

example_nr = 12
matplotlib.pyplot.imshow(raw_train_images[example_nr], cmap=matplotlib.pyplot.cm.binary)
print("Label:", raw_train_labels[example_nr])

## Baseline-Modell

Wir starten nun mit einem sehr einfachen neuronalen Netzwerk als Referenz, das nur einen einzigen "hidden layer" mit 512 Neuronen enthält.

In [ ]:
import keras.models
import keras.layers

network = keras.models.Sequential()
network.add(keras.layers.Flatten(input_shape=(28, 28)))
network.add(keras.layers.Dense(32, activation="relu"))
network.add(keras.layers.Dense(10, activation="softmax"))

network.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
network.summary()

-> Wie man sieht, besteht das Netzwerk nun aus drei Schichten: Input, Hidden Layer und Output.

### Datenaufbereitung

Wie man sieht, besitzt die Ausgabeschicht 10 Neuronen (Output Shape = (None, 10) in der Zusammenfassung). Bei neuronalen Netzen ist es allgemein üblich, dass man bei Klassifizierungsaufgaben soviele Ausgabeneuronen nutzt, wie es Klassen gibt. Effektiv gibt es für jede Ziffer ein Ausgabeneuron, das sich umso stärker meldet, je sicherer es ist, "seine" Klasse erkannt zu haben.

Das bedeutet, dass wir die Daten noch umformen müssen, damit die Trainingsdaten auch diese Struktur erhalten. Konkret sieht die Umformung wie folgt aus:

```
0 -> (1, 0, 0, 0, 0, 0, 0, 0, 0, 0)
1 -> (0, 1, 0, 0, 0, 0, 0, 0, 0, 0)
2 -> (0, 0, 1, 0, 0, 0, 0, 0, 0, 0)
```
usw.

Diese Transformation heißt "One-Hot" Encoding, da immer genau ein Element des Vektors gefüllt ist.

In [ ]:
import keras.utils

train_labels = keras.utils.to_categorical(raw_train_labels)

Wir überprüfen, ob alles so geklappt hat wie beschrieben. Auch hier solltet ihr ein paar andere Beispiele betrachten, indem ihr den Wert von `example_nr` anpasst.

In [ ]:
example_nr = 42
print(raw_train_labels[example_nr], "->", train_labels[example_nr])

Zusätzlich normieren wir noch den Bereich der Grauwerte von 0 bis 255 (Integer-Werte) auf den Bereich 0 bis 1 (Gleitkommazahlen), da das neuronale Netz intern mit Gleitkommazahlen arbeitet.

In [ ]:
train_images = raw_train_images.astype("float32") / 255

### Training

In [ ]:
import numpy

# Setzen der Startwerte für den beim Training benutzten Zufallszahlengenerator
tensorflow.set_random_seed(4242)
numpy.random.seed(4242)

network.fit(train_images, train_labels, epochs=5, batch_size=128)

Das Training läuft (bedingt durch die einfache Architektur) sehr schnell. Trotzdem erreichen wir schon eine Genauigkeit von respektablen 95% auf den Trainingsdaten.

### Auswertung

Nun überprüfen wir, ob wir auf den Testdaten ähnlich gute Ergebnisse erzielen. Ein Abfall der Güte würde zeigen, dass ein Overfitting vorliegt.

In [ ]:
test_images = raw_test_images.astype("float32") / 255
test_labels = keras.utils.to_categorical(raw_test_labels)

In [ ]:
baseline_loss, baseline_accuracy = network.evaluate(test_images, test_labels)
baseline_loss, baseline_accuracy

-> Etwa 95% der Test-Ziffern wurden korrekt klassifiziert (bei einem Loss von ca. 0.15). Damit haben wir die Genauigkeit auch auf dem Testset bestätigt.

### Tuning

Wir machen jetzt sogenanntes "Hyperparameter-Tuning", d.h. wir verändern gezielt Parameter der Architektur und des Lernverfahrens, um eine möglichst hohe Genauigkeit zu erzielen.

Bei unserem Modell oben hatten wir 32 Neuronen für den Hidden Layer gewählt. Das war eine willkürliche Wahl. Wir probieren jetzt systematisch unterschiedliche Größen (einige Zweipotenzen zwischen 2 und 4096) für die Anzahl der Neuronen, um zu prüfen, für welche Anzahl wir die besten Ergebnisse bekommen.

In [ ]:
import math


# Reduktion der Trainingsdaten um den Effekt bei geringer Trainingsdauer sichtbar zu machen
train_images_subset = train_images[:30000]
train_labels_subset = train_labels[:30000]


# Definition eines Arbeitsschrittes, um zu gegebener Netzwerkgröße die Kosten auszurechnen
def loss(hidden_unit_count):
    tensorflow.set_random_seed(4242)
    numpy.random.seed(4242)
    network = keras.models.Sequential()
    network.add(keras.layers.Flatten(input_shape=(28, 28)))
    network.add(keras.layers.Dense(hidden_unit_count, activation="relu"))
    network.add(keras.layers.Dense(10, activation="softmax"))
    network.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    
    epochs = 2 + int(math.log(hidden_unit_count, 16) ** 2)
    network.fit(train_images_subset, train_labels_subset, epochs=epochs, batch_size=128, verbose=False)
    
    print(
        f"with {hidden_unit_count:5} hidden units: ",
        end=''
    )
    
    train_loss, train_accuracy = network.evaluate(train_images_subset, train_labels_subset, verbose=False)
    test_loss, test_accuracy = network.evaluate(test_images, test_labels, verbose=False)
    
    print(
        f"training accuracy={train_accuracy:.1%}, loss={train_loss:.4f}, "
        f"test accuracy={test_accuracy:.1%}, loss={test_loss:.4f}"
    )
    return train_accuracy, test_accuracy, train_loss, test_loss

# Berechnen der Kosten für verschiedene Netzwerkgrößen
hidden_unit_counts = [2, 4, 16, 64, 128, 512, 1024, 4096]
%time losses = [loss(x) for x in hidden_unit_counts]
train_accuracies, test_accuracies, train_losses, test_losses = list(zip(*losses))

-> Wie man sieht, nimmt die Genauigkeit mit einer zunehmen Anzahl an Neuronen zu. Ganz am Ende sieht man jedoch, dass für das Validation Set der Loss wieder zunimmt und die Genauigkeit sinkt. Damit sind wir im Bereich des Overfittings.

Wir stellen die Ergebnisse nun graphisch dar.

In [ ]:
# Plotten der Ergebnisse

matplotlib.pyplot.plot(hidden_unit_counts, train_losses, ":g", label="training")
matplotlib.pyplot.plot(hidden_unit_counts, test_losses, "g", label="test")
matplotlib.pyplot.xscale("log")
matplotlib.pyplot.ylabel("loss")
matplotlib.pyplot.axhline(baseline_loss, color="b", linestyle="--", label="baseline")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()

matplotlib.pyplot.plot(hidden_unit_counts, train_accuracies, ":g", label="training")
matplotlib.pyplot.plot(hidden_unit_counts, test_accuracies, "g", label="test")
matplotlib.pyplot.xscale("log")
matplotlib.pyplot.xlabel("hidden units")
matplotlib.pyplot.ylabel("accuracy")
matplotlib.pyplot.axhline(baseline_accuracy, color="b", linestyle="--", label="baseline")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()

Wir zoomen noch etwas weiter in den Plot rein, um den Effekt des Overfittings besser auflösen zu können:

In [ ]:
matplotlib.pyplot.plot(hidden_unit_counts, train_accuracies, ":g", label="training")
matplotlib.pyplot.plot(hidden_unit_counts, test_accuracies, "g", label="test")
matplotlib.pyplot.xscale("log")
matplotlib.pyplot.xlabel("hidden units")
matplotlib.pyplot.ylabel("loss")
matplotlib.pyplot.ylim(0.9, 1)
matplotlib.pyplot.axhline(baseline_accuracy, color="b", linestyle="--", label="baseline")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()

Auf dieser Basis würden wir einen Wert um die 4096 Neuronen für unser finales Modell wählen.

## Alternatives Modell

Zum Abschluss erproben wir nun einen noch deutlich leistungsfähigeren Ansatz, ein sogenanntes "Convolutional Neural Network". Dieses ist speziell an die 2D-Struktur von Bilddaten angepasst.

In [ ]:
convnet = keras.models.Sequential()
convnet.add(keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)))
convnet.add(keras.layers.MaxPooling2D((2, 2)))
convnet.add(keras.layers.Conv2D(64, (3, 3), activation="relu"))
convnet.add(keras.layers.MaxPooling2D((2, 2)))
convnet.add(keras.layers.Conv2D(64, (3, 3), activation="relu"))
convnet.add(keras.layers.Dropout(0.25))
convnet.add(keras.layers.Flatten())
convnet.add(keras.layers.Dense(64, activation="relu"))
convnet.add(keras.layers.Dropout(0.5))
convnet.add(keras.layers.Dense(10, activation="softmax"))

convnet.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
convnet.summary()

-> Wie man sieht, ist dieses Netzwerk vom Aufbau auch schon deutlich komplizierter.

Wir trainieren das Netzwerk nun auf unseren Daten.

In [ ]:
# Setzen der Startwerte für den beim Training benutzten Zufallszahlengenerator
tensorflow.set_random_seed(4242)
numpy.random.seed(4242)

convnet.fit(
    numpy.expand_dims(train_images, axis=-1),
    train_labels,
    epochs=5,
    batch_size=64
)

Nun überprüfen wir die Genauigkeit auf den Testdaten:

In [ ]:
convnet_loss, convnet_accuracy = convnet.evaluate(
    numpy.expand_dims(test_images, -1),
    test_labels
)
convnet_accuracy

Wir erreichen eine Genauigkeit von sehr guten 99,2%. Wir fügen dieses Ergebnis in die Abbildung von oben ein, zusammen mit dem aktuell besten Modell aus der Forschung ('dropconnect').

In [ ]:
matplotlib.pyplot.plot(hidden_unit_counts, test_accuracies, "g", label="naive net")
matplotlib.pyplot.xscale("log")
matplotlib.pyplot.xlabel("hidden units")
matplotlib.pyplot.ylabel("accuracy")
matplotlib.pyplot.ylim(.95, 1.0)
matplotlib.pyplot.axhline(baseline_accuracy, color="b", linestyle="--", label="baseline")
matplotlib.pyplot.axhline(convnet_accuracy, color="c", linestyle="-", label="convnet")
matplotlib.pyplot.axhline(0.9979, color="r", linestyle="--", label="dropconnect")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()

Wie man sieht, ist das Convolutional Neural Network ("convnet") auch deutlich besser als unser bestes bisheriges Netzwerk. Auch hier könnte man natürlich wieder die Hyperparameter optimieren, um z.B. die Größe der verschiedenen Zwischenschichten anzupassen.

### Diagnostik

Wir geben uns jetzt die aus Sicht unser besten Modells 'perfekten' Ziffer-Bilder aus. In diesem Fall hier kann man sie als als "Durchschnitt" über alle entsprechenden Ziffern in den Trainingsdaten interpretieren.

In [ ]:
import vis.visualization
import vis.utils

convnet_visualization = keras.models.clone_model(convnet)
convnet_visualization.layers[-1].activation = keras.activations.linear
convnet_visualization = vis.utils.utils.apply_modifications(convnet_visualization)
convnet_visualization.set_weights(convnet.get_weights())

for i in range(10):
    matplotlib.pyplot.title(f"{i}")
    matplotlib.pyplot.imshow(
        vis.visualization.visualize_activation(
            convnet_visualization,
            layer_idx=-1,
            filter_indices=i,
            tv_weight=10.0,
            lp_norm_weight=0.0,
            input_range=(0., 1.)
        )[..., 0],
        cmap=matplotlib.pyplot.cm.binary,
    )
    matplotlib.pyplot.show()

Schließlich können wir wieder analog zum Hunde & Katzen Modell aus dem Einführungsvortrag ausgeben lassen, auf welche Bereiche des Bildes das Modell abhängig von der möglichen Vorhersageziffer schaut. Mit ein bischen Phantasie kann man sehen, dass das Modell z.B. bei der Beurteilung, ob es sich um eine 3 handelt, nur auf die Bereiche schaut, die bei einer 3 schwarz wären.

Ihr könnt gerne die Variable `example_nr` anders belegen und experimentieren!

In [ ]:
def plot_attention(image):
    matplotlib.pyplot.figure(figsize=(20, 10))
    matplotlib.pyplot.subplot(3, 4, 1)
    matplotlib.pyplot.imshow(image, cmap=matplotlib.pyplot.cm.binary)
    
    for kind in range(10):
        matplotlib.pyplot.subplot(3, 4, kind + 2)
        matplotlib.pyplot.imshow(
            vis.visualization.visualize_saliency(
                convnet_visualization,
                layer_idx=-1,
                filter_indices=kind,
                backprop_modifier="guided",
                seed_input=image.reshape(28, 28, 1),
            ),
            cmap=matplotlib.pyplot.cm.jet
        )
        matplotlib.pyplot.title(kind)
    
    prediction = convnet.predict(image.reshape(1, 28, 28, 1))[0]
    matplotlib.pyplot.subplot(3, 4, 12)
    matplotlib.pyplot.bar(range(10), prediction, tick_label=range(10))
    matplotlib.pyplot.title(numpy.argmax(prediction))
    
    matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.show()
    
    return prediction

example_nr = 1337

plot_attention(test_images[example_nr])

In [ ]:
pred = numpy.argmax(convnet.predict(numpy.expand_dims(test_images, axis=-1)), axis=1)
pred_errors = numpy.nonzero(pred != raw_test_labels)[0]

for test_nr in pred_errors:
    matplotlib.pyplot.imshow(raw_test_images[test_nr], cmap=matplotlib.pyplot.cm.binary)
    matplotlib.pyplot.show()
    print("Label:", raw_test_labels[test_nr], "Prediction:", pred[test_nr], end="\n\n")